# Favorita Sales Forecasting


In [ ]:
# Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Load Data
dataset = pd.read_csv('dataset.csv', parse_dates=['date'])
validations = pd.read_csv('validations.csv', parse_dates=['date'])
stores = pd.read_csv('stores.csv')
oil = pd.read_csv('oil.csv', parse_dates=['date'])
holidays = pd.read_csv('holidays_events.csv', parse_dates=['date'])
transactions = pd.read_csv('transactions.csv', parse_dates=['date'])

In [ ]:
# Merge Additional Data
data = dataset.merge(stores, on='store_nbr', how='left')
data = data.merge(oil, on='date', how='left')
data = data.merge(transactions, on=['date', 'store_nbr'], how='left')

# Clean holidays data
holidays = holidays[holidays.transferred == False]
holidays['is_holiday'] = 1
data = data.merge(holidays[['date', 'is_holiday']], on='date', how='left')
data['is_holiday'] = data['is_holiday'].fillna(0)

In [ ]:
# Feature Engineering
data['day'] = data['date'].dt.day
data['month'] = data['date'].dt.month
data['year'] = data['date'].dt.year
data['weekday'] = data['date'].dt.weekday
data['onpromotion'] = data['onpromotion'].fillna(0)

In [ ]:
# Encode categorical features
data['family'] = data['family'].astype('category').cat.codes
data['city'] = data['city'].astype('category').cat.codes
data['state'] = data['state'].astype('category').cat.codes
data['type'] = data['type'].astype('category').cat.codes

In [ ]:
# Model Training
features = ['store_nbr', 'family', 'onpromotion', 'day', 'month', 'year', 'weekday', 'dcoilwtico', 'transactions', 'is_holiday', 'city', 'state', 'type', 'cluster']
X = data[features]
y = data['sales']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=False)
model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05)

from lightgbm import early_stopping, log_evaluation

model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.05)

model.fit(
    X_train,
    y_train,
    eval_set=[(X_test, y_test)],
    callbacks=[
        early_stopping(stopping_rounds=10),
        log_evaluation(period=10)
    ]
)



[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.012960 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 540
[LightGBM] [Info] Number of data points in the train set: 242632, number of used features: 12
[LightGBM] [Info] Start training from score 200.810477
Training until validation scores don't improve for 10 rounds
[10]	valid_0's l2: 178957
[20]	valid_0's l2: 87025.2
[30]	valid_0's l2: 50544
[40]	valid_0's l2: 34642.8
[50]	valid_0's l2: 26916.6
[60]	valid_0's l2: 23381
[70]	valid_0's l2: 21073.8
[80]	valid_0's l2: 19561.4
[90]	valid_0's l2: 18611.8
[100]	valid_0's l2: 17982.7
Did not meet early stopping. Best iteration is:
[100]	valid_0's l2: 17982.7


LGBMRegressor(learning_rate=0.05)

In [ ]:
# Prepare Validation Set
valid = validations.merge(stores, on='store_nbr', how='left')
valid = valid.merge(oil, on='date', how='left')
valid = valid.merge(transactions, on=['date', 'store_nbr'], how='left')
valid = valid.merge(holidays[['date', 'is_holiday']], on='date', how='left')
valid['is_holiday'] = valid['is_holiday'].fillna(0)
valid['day'] = valid['date'].dt.day
valid['month'] = valid['date'].dt.month
valid['year'] = valid['date'].dt.year
valid['weekday'] = valid['date'].dt.weekday
valid['family'] = valid['family'].astype('category').cat.codes
valid['city'] = valid['city'].astype('category').cat.codes
valid['state'] = valid['state'].astype('category').cat.codes
valid['type'] = valid['type'].astype('category').cat.codes

In [ ]:
# Make Predictions
valid_X = valid[features]
valid['sales'] = model.predict(valid_X)
submission = pd.DataFrame({
    'id': valid.index + 3000888,  # starting id
    'sales': valid['sales']
})
submission.to_csv('submission.csv', index=False)

submission.head()


id      sales
0  3000888 -16.818055
1  3000889 -16.818055
2  3000890 -16.818055
3  3000891  31.132688
4  3000892   2.071308

In [ ]:
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')